![Erudio logo](../../img/erudio-logo-small.png)

# Contextlib and Advanced Context Managers

In the last lesson you saw the Context Manager Protocol.  However, much as with iterators that are usually expressed much more easily as generator functions than as classes, the same it true of context managers.  By using the module `contextlib`, you can express context managers in a compact, flexible, and intuitive manner.

As a first simple example, let us replicate the very simple `MyContext()` class from the last lesson, even more simply.

In [1]:
from contextlib import contextmanager

@contextmanager
def my_context(val=42):
    try:
        print("Initializing context")
        yield val
    except BaseException as err:
        print(f"CM body raised {err.__class__.__name__}({err.args[0]})")
        print(f"Inspect {err.__traceback__}")
    finally:
        print("Exiting context")

In [2]:
with my_context() as t:
    print("The answer is", t)

Initializing context
The answer is 42
Exiting context


In [3]:
with my_context(33) as t:
    x = t/0
    print(x)

Initializing context
CM body raised ZeroDivisionError(division by zero)
Inspect <traceback object at 0x00000215FF102180>
Exiting context


## Temporary settings

Being able to bind an integer value isn't that impressive.  We could yield any other (i.e. more complex and interesting) context object as well.  But often what we would like to do is set something up and tear it down.  We may not even need any context object to do that.

In [4]:
# Some global configuration information
settings = {'host': "ine.com", 'port': 80, 
            'user': 'JohnColtrane',
            'password': 'my_favorite_things'}

In [5]:
@contextmanager
def configure(**kws):
    global settings
    try:
        old_settings = settings.copy()
        settings.update(kws)
        yield
    finally:
        settings = old_settings

We can temporarily utilize some different settings.  Notice that here we do not bother binding any context object.

In [6]:
with configure(host='localhost', port=70):
    print("Connecting to host:", settings['host'])
    print("Using port:", settings['port'])

Connecting to host: localhost
Using port: 70


In [7]:
settings

{'host': 'ine.com',
 'port': 80,
 'user': 'JohnColtrane',
 'password': 'my_favorite_things'}

# The "dual" of subroutines

The most common way users think about context managers is as a way to allocate some resources, run some user code with those resources in place, then guarantee cleanup afterwards.  This use is definitely useful, but context managers are much more general when thought of as a flow-control construct.

The fundamental construct of [Structured Programming](https://en.wikipedia.org/wiki/Structured_programming) is organizing flow control in terms of calls to subroutines that contain repeated code (also called `functions` or `procedures` or `methods` by some programming languages).  All modern procedural programming (including object oriented programming) relies on this abstraction.

We can think of context managers as another construct that is something like the mathematical <a href="https://en.wikipedia.org/wiki/Duality_(mathematics)">dual</a> of subroutines, and that are in some sense almost as general as a construct.

## Reusing the middle

What a subroutine does is to insert—by named reference—a block of reusable code in the middle of some lines of local-use code.  That local use might live inside a reusable block, but a call is always an encapsulation of functionality relative to the context.  The code we write every day looks something like this:

```python
def do_some_things(a, b, c, d, e):
    "Some sort of numeric steps to get a result"
    
    #---- The Prelude -----#
    a1 = a + b * c       # This line only relevant within this function
    for i in range(10):  # This whole loop also specific to this context
        b1 = b - a * d   
        if b1 > 0:
            b = e % b1
            
    #----- The reusable code, suitably parameterized -----#       
    x = encapsulated_code(a1, b1, 10)
        
    #----- The Finalization -----#
    w = x + a1 + b
    y = math.log(w + x)
    z = y**2
    return z
```

We can imagine that the function `encapsulated_code()` has many steps inside of it, similar to `do_some_things()` itself.  It might have assignments, conditionals, branches, operations on values, etc.  But the idea is simply that rather repeating those multiple lines of code within the three branches in the playful example, we can simply call them with a meaningful name.  Moreover, many other functions not shown might equally take advantage of `encapsulated_code()` and whatever calculation it performs.

Technically, we also call another function, `math.log()` during the final steps, and obviously that operation incorporates many steps that we would rather not write every time we want the logarithm of a number.

## Reusing the margins

Take this very familiar concept of subroutines, and so-to-speak, "turn it inside out."  Rather than having a block of reusable code that we utilize in the middle of our own local code, imagine that the reusable code acts as a wrapper around the outsides—or *margins*—of the code that we want to write for custom purposes.

Clearly all those programming languages that do not have context managers manage to accomplish.  In some cases, something similar can be done with object constructors and destructors.  But more often, programmers simply write their "prelude" and "finalize" as separate functions and call those at the ends of their local purpose code:

```python
def enter_and_exit(a, b, c):
    prelude(...)
    # ... various lines of code for this local purpose
    finalize(...)
```

If various preludes and finalizations might be relevant to "bookend" local code, the style shown is exactly what is needed.  However, it often happens that the prelude and finalization are closely tied to each other, and you always want them to match up in bookending your code.  When that occurs, allowing a single construct to define such "bookend" code is nice to have.

## Some moderately real-world numeric code

The premise in the below examples are that we might want to do a variety of operations upon 2-D NumPy arrays, but after completing our local operations, we want to "squeeze" the results back towards the means of each column of data values.  This exact operation may not have any real world purpose, but the idea of consistent post-processing of arrays across a range of custom transformations is applicable to a variety of tasks.

The key here is that we need to determine some derived property of the *original* array to use in applying a transformation to the locally transformed array.  If that derived property and the post-processing are consistently used together, factoring them as a context manager is a good approach.

In [8]:
import numpy as np
from contextlib import contextmanager

@contextmanager
def squeeze_to_mean(arr):
    assert arr.ndim == 2
    mean = arr.mean(axis=0)
    # This is the thing bound by the 'as'
    yield mean
    # Your code lives here when used
    arr[:] = (arr + mean)/2

Within the `squeeze_to_mean` context we do a few operations on our original array, some of which are based on the column means provided by the context manager.  In the examples, we simply add a little bit of random jitter to the values and print off some intermediate array values.

In [9]:
np.random.seed(0)
arr = np.arange(20, dtype=float).reshape(5, 4)
print('arr:')
print(arr)

with squeeze_to_mean(arr) as mean:
    jitter = mean/2 * np.random.random(20).reshape(5,4) - 0.5
    arr[:] += jitter
    print("\njittered-only array:")
    print(arr)
    arr[np.abs(arr-10) >= 4] = 10
    print("\njittered array with outliers set to 10:")
    print(arr)

print("\nmean along axis 0:")
print(mean)

print("\njittered array pulled towards mean:")
print(arr)

arr:
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]
 [16. 17. 18. 19.]]

jittered-only array:
[[ 1.69525402  3.71835215  4.51381688  5.49685751]
 [ 5.1946192   7.40652351  7.68793606 11.4047515 ]
 [11.35465104 10.22548683 13.45862519 13.40892206]
 [13.77217824 16.66518487 13.85518029 14.97921115]
 [15.58087359 20.2467893  21.39078375 23.28506682]]

jittered array with outliers set to 10:
[[10.         10.         10.         10.        ]
 [10.          7.40652351  7.68793606 11.4047515 ]
 [11.35465104 10.22548683 13.45862519 13.40892206]
 [13.77217824 10.         13.85518029 10.        ]
 [10.         10.         10.         10.        ]]

mean along axis 0:
[ 8.  9. 10. 11.]

jittered array pulled towards mean:
[[ 9.          9.5        10.         10.5       ]
 [ 9.          8.20326175  8.84396803 11.20237575]
 [ 9.67732552  9.61274342 11.7293126  12.20446103]
 [10.88608912  9.5        11.92759015 10.5       ]
 [ 9.          9.5        10.         10.5   

# Exercises

## Description

Almost all of the Python standard library tools that can benefit from exposing themselves as a context manager already do.  For example `zipfile.ZipFile` provides such a CM interface.  However, for this exercise, we would like a slightly different context manager interface.

Your context manager should be configured with two different names, in "input" zip file and an "output" zipfile.  The context object provided should be a list of filenames extracted from `input.zip` that are placed into a temporary directory `tmp-zip-ops/` to avoid stepping on any names that might already exist in your working directory.  After the body of your context performs whatever modifications of the files you wish, all the files from `input.zip` (in potentially modified form) should be placed into the `output.zip` archive, and the temporary directory should be removed to cleanup.  For this exercise, `output.zip` should be overwritten if it already exists.

For example:

```python
with zipmod(input='this.zip', output='that.zip') as fnames:
    for fname in fnames:
        with open(fname, 'a') as fh:
            print("\nMORE STUFF", file=fh)
```

## Setup

In [10]:
import zipfile
import contextlib

zipmod = "A context manager to implment"

## Solution

In [17]:
import glob
from pathlib import Path
import shutil
import os

@contextlib.contextmanager
def zipmod(input='input.zip', output='output.zip'):
    try:
        path = Path('tmp-zip-ops')
        print(path)
        os.makedirs(path, exist_ok=True)
        with zipfile.ZipFile(input) as zfin:
            zfin.extractall(path)
        fnames = path.glob('*')
        yield fnames
        with zipfile.Zipfile(output, 'w') as zfout:
            for fname in fnames:
                zfout.write(fname)
    finally:
        shutil.rmtree(path)

## Test Cases

In [ ]:
def test_this():
    with zipmod(input='this.zip', output='that.zip') as fnames: # substitute the names of the zip files as per your test case
        for fname in fnames:
            with open(fname, 'a') as fh:
                print("\nMORE STUFF", file=fh)
        assert 2 == 3
    
test_this()

In [4]:
def test_that():
    assert 1 == 2
    
test_that()

AssertionError: 

-------------
Materials licensed under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/) by the authors